<h3>This notebook are are load data and cleaning all the nan values then convert categorial columns in
numeric values</h3>

In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def read_data(path):
    '''
    this function will read the txt file which is a line-delimited json file and produce a pandas dataframe.
    
    '''
    my_list =[]
    with open(path) as f:
        for line in f:
            json_content = json.loads(line)
            my_list.append(json_content)
    #df = pd.DataFrame(my_list) 
    return(pd.DataFrame(my_list))

<h5>Lets read dataset and cleanup missing value </h5>

In [3]:
df = read_data('../data/transactions.txt')
# Replacing blank values with nan.
df.replace(r'^\s*$', np.nan, regex=True, inplace=True) 
# lets inspect missing values now
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641914 entries, 0 to 641913
Data columns (total 29 columns):
accountNumber               641914 non-null object
accountOpenDate             641914 non-null object
acqCountry                  638001 non-null object
availableMoney              641914 non-null float64
cardCVV                     641914 non-null object
cardLast4Digits             641914 non-null object
cardPresent                 641914 non-null bool
creditLimit                 641914 non-null float64
currentBalance              641914 non-null float64
currentExpDate              641914 non-null object
customerId                  641914 non-null object
dateOfLastAddressChange     641914 non-null object
echoBuffer                  0 non-null float64
enteredCVV                  641914 non-null object
expirationDateKeyInMatch    641914 non-null bool
isFraud                     641914 non-null bool
merchantCategoryCode        641914 non-null object
merchantCity                0

#### Observations:

There are 641914 instances in the dataset. We have some numerical attributes like 'availableMoney', 'creditLimit', and some categorical attributes like 'merchantCategoryCode', 'merchantName'. We have few attributes which totally have missing values. These attributes are:

- echoBuffer  
- merchantCity
- merchantState
- merchantZip
- posOnPremises
- recurringAuthInd
 

We can drop these columns.

In [4]:
df_clean = df.drop(['echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 
                 'posOnPremises','recurringAuthInd'], axis=1)

#### Handeling missing values

We have few options:

- totally drop those attributes from data.
- Drop those records (remove rows where these attributes are missing)
- Set the missing to some values. For numerical attributes, we can set them to the mean/median, and for categorical attributes we can set them to the most frequent category.

but this case we are dropping all the row that has missing value

In [5]:
df_clean = df_clean.dropna()

Now let convert <b>isFraud</b> to numberic value.


In [6]:
df_clean['isFraud'] = df_clean['isFraud'].apply(lambda x: 1 if x == True else 0)

#### Handle date and datetime

Different systems stores dates in different formats: 11.12.2019, 2016-02-12, Sep 24, 2003 etc. But for building models on dates data, we need to somehow convert it to a numeric format. There are 3 most common methods to transform date to numeric format: Unix timestamp , KSP date format , Divide into several features ,Construct new feature For this dataset , the third method is used 'Divide into several features ' because :

perfectly preserve intervals easy intuition and knowledge discovery more dimensions you add, more complex your model could get, but it is not always bad.


#### Convert our date columns to datetime


In [7]:
## temporally remove date column
date_columns=['accountOpenDate', 'transactionDateTime', 'currentExpDate' , 'dateOfLastAddressChange']
df_date_columns = df_clean[date_columns]

## convert this columns to datetime
df_date_columns['accountOpenDate'] = pd.to_datetime(df_date_columns['accountOpenDate'], format = "%Y-%m-%d")
df_date_columns['transactionDateTime'] = pd.to_datetime(df_date_columns['transactionDateTime'], format = "%Y-%m-%dT%H:%M:%S")
df_date_columns['currentExpDate'] = pd.to_datetime(df_date_columns['currentExpDate'], format = "%m/%Y")
df_date_columns['dateOfLastAddressChange'] = pd.to_datetime(df_date_columns['dateOfLastAddressChange'], format = "%Y-%m-%d")
df_date_columns.head()

,accountOpenDate,transactionDateTime,currentExpDate,dateOfLastAddressChange
0,2014-08-03,2016-01-08 19:04:50,2020-04-01,2014-08-03
1,2014-08-03,2016-01-09 22:32:39,2023-06-01,2014-08-03
2,2014-08-03,2016-01-11 13:36:55,2027-12-01,2014-08-03
3,2014-08-03,2016-01-11 22:47:46,2029-09-01,2014-08-03
4,2014-08-03,2016-01-16 01:41:11,2024-10-01,2014-08-03


In [8]:
# work on accountOpenDate and split into three component year month and day
df_date_columns["accountOpenDate_year"] = [t.year for t in df_date_columns['accountOpenDate']]
df_date_columns["accountOpenDate_month"] = [t.month for t in df_date_columns['accountOpenDate']]
df_date_columns["accountOpenDate_day"] = [t.day for t in df_date_columns['accountOpenDate']]

# work on transactionDateTime open date and split into three component year month  day hh min and ss
df_date_columns["transactionDateTime_year"] = [t.year for t in df_date_columns['transactionDateTime']]
df_date_columns["transactionDateTime_month"] = [t.month for t in df_date_columns['transactionDateTime']]
df_date_columns["transactionDateTime_day"] = [t.day for t in df_date_columns['transactionDateTime']]

df_date_columns["transactionDateTime_hour"] = [t.hour for t in df_date_columns['transactionDateTime']]
df_date_columns["transactionDateTime_minute"] = [t.minute for t in df_date_columns['transactionDateTime']]
df_date_columns["transactionDateTime_second"] = [t.second for t in df_date_columns['transactionDateTime']]

# work on currentExpDate and split into three component year month and day
df_date_columns["currentExpDate_year"] = [t.year for t in df_date_columns['currentExpDate']]
df_date_columns["currentExpDate_month"] = [t.month for t in df_date_columns['currentExpDate']]
df_date_columns["currentExpDate_day"] = [t.day for t in df_date_columns['currentExpDate']]

# work on dateOfLastAddressChange and split into three component year month and day
df_date_columns["dateOfLastAddressChange_year"] = [t.year for t in df_date_columns['dateOfLastAddressChange']]
df_date_columns["dateOfLastAddressChange_month"] = [t.month for t in df_date_columns['dateOfLastAddressChange']]
df_date_columns["dateOfLastAddressChange_day"] = [t.day for t in df_date_columns['dateOfLastAddressChange']]

#drop all the original date colums from date dataframe
df_date_columns = df_date_columns.drop(date_columns, axis=1)
df_date_columns.head()

,accountOpenDate_year,accountOpenDate_month,accountOpenDate_day,transactionDateTime_year,transactionDateTime_month,transactionDateTime_day,transactionDateTime_hour,transactionDateTime_minute,transactionDateTime_second,currentExpDate_year,currentExpDate_month,currentExpDate_day,dateOfLastAddressChange_year,dateOfLastAddressChange_month,dateOfLastAddressChange_day
0,2014,8,3,2016,1,8,19,4,50,2020,4,1,2014,8,3
1,2014,8,3,2016,1,9,22,32,39,2023,6,1,2014,8,3
2,2014,8,3,2016,1,11,13,36,55,2027,12,1,2014,8,3
3,2014,8,3,2016,1,11,22,47,46,2029,9,1,2014,8,3
4,2014,8,3,2016,1,16,1,41,11,2024,10,1,2014,8,3


In [9]:
# remove categorial columns from df_clean and concat df_date_clean
df_clean = df_clean.drop(date_columns, axis=1)
df_clean = pd.concat([df_clean, df_date_columns], axis=1)
df_clean.head()

,accountNumber,acqCountry,availableMoney,cardCVV,cardLast4Digits,cardPresent,creditLimit,currentBalance,customerId,enteredCVV,...,transactionDateTime_day,transactionDateTime_hour,transactionDateTime_minute,transactionDateTime_second,currentExpDate_year,currentExpDate_month,currentExpDate_day,dateOfLastAddressChange_year,dateOfLastAddressChange_month,dateOfLastAddressChange_day
0,733493772,US,5000.00,492,9184,False,5000.0,0.00,733493772,492,...,8,19,4,50,2020,4,1,2014,8,3
1,733493772,US,4888.67,492,9184,False,5000.0,111.33,733493772,492,...,9,22,32,39,2023,6,1,2014,8,3
2,733493772,US,4863.92,492,9184,False,5000.0,136.08,733493772,492,...,11,13,36,55,2027,12,1,2014,8,3
3,733493772,US,4676.52,492,9184,False,5000.0,323.48,733493772,492,...,11,22,47,46,2029,9,1,2014,8,3
4,733493772,US,4449.18,492,9184,False,5000.0,550.82,733493772,492,...,16,1,41,11,2024,10,1,2014,8,3


#### Now we are converting categorial column to numeric using LabelEncoder

In [10]:
# instantiate labelencoder object
le = LabelEncoder()

In [11]:
category_columns =['acqCountry','cardPresent','expirationDateKeyInMatch','isFraud','merchantCategoryCode',
          'merchantCountryCode','merchantName','posConditionCode','posEntryMode','transactionType']

In [12]:
df_clean.isna().sum()

accountNumber                    0
acqCountry                       0
availableMoney                   0
cardCVV                          0
cardLast4Digits                  0
cardPresent                      0
creditLimit                      0
currentBalance                   0
customerId                       0
enteredCVV                       0
expirationDateKeyInMatch         0
isFraud                          0
merchantCategoryCode             0
merchantCountryCode              0
merchantName                     0
posConditionCode                 0
posEntryMode                     0
transactionAmount                0
transactionType                  0
accountOpenDate_year             0
accountOpenDate_month            0
accountOpenDate_day              0
transactionDateTime_year         0
transactionDateTime_month        0
transactionDateTime_day          0
transactionDateTime_hour         0
transactionDateTime_minute       0
transactionDateTime_second       0
currentExpDate_year 

In [13]:
df_clean[category_columns] = df_clean[category_columns].apply(lambda col: le.fit_transform(col))
df_clean.head()

,accountNumber,acqCountry,availableMoney,cardCVV,cardLast4Digits,cardPresent,creditLimit,currentBalance,customerId,enteredCVV,...,transactionDateTime_day,transactionDateTime_hour,transactionDateTime_minute,transactionDateTime_second,currentExpDate_year,currentExpDate_month,currentExpDate_day,dateOfLastAddressChange_year,dateOfLastAddressChange_month,dateOfLastAddressChange_day
0,733493772,3,5000.00,492,9184,0,5000.0,0.00,733493772,492,...,8,19,4,50,2020,4,1,2014,8,3
1,733493772,3,4888.67,492,9184,0,5000.0,111.33,733493772,492,...,9,22,32,39,2023,6,1,2014,8,3
2,733493772,3,4863.92,492,9184,0,5000.0,136.08,733493772,492,...,11,13,36,55,2027,12,1,2014,8,3
3,733493772,3,4676.52,492,9184,0,5000.0,323.48,733493772,492,...,11,22,47,46,2029,9,1,2014,8,3
4,733493772,3,4449.18,492,9184,0,5000.0,550.82,733493772,492,...,16,1,41,11,2024,10,1,2014,8,3


In [14]:
#Check last time is there any nan
df_clean.isna().sum()

accountNumber                    0
acqCountry                       0
availableMoney                   0
cardCVV                          0
cardLast4Digits                  0
cardPresent                      0
creditLimit                      0
currentBalance                   0
customerId                       0
enteredCVV                       0
expirationDateKeyInMatch         0
isFraud                          0
merchantCategoryCode             0
merchantCountryCode              0
merchantName                     0
posConditionCode                 0
posEntryMode                     0
transactionAmount                0
transactionType                  0
accountOpenDate_year             0
accountOpenDate_month            0
accountOpenDate_day              0
transactionDateTime_year         0
transactionDateTime_month        0
transactionDateTime_day          0
transactionDateTime_hour         0
transactionDateTime_minute       0
transactionDateTime_second       0
currentExpDate_year 

#### Save clean data

In [15]:
df_clean.to_csv('../data/processed_transaction.csv', index=False)

### Code Refercence

This notebook is combined from experimental notebooks. Where Every body work on different task on different prepossing part. here is our team member reference notebook

https://github.com/msarker000/ml-group-project/blob/master/experimental-notebooks/Experimental_handle_date.ipynb --mary
https://github.com/msarker000/ml-group-project/blob/master/experimental-notebooks/try%20labelEncoder%20and%20OneHotEncoder%20for%20encoding%20categorical%20features.ipynb --marjan
https://github.com/msarker000/ml-group-project/blob/master/experimental-notebooks/experimental-data-prepossessing-notebook.ipynb -- ayub